In [ ]:
import scipy.io
import numpy as np
from skimage import io
from PIL import Image
import pandas as pd
from random import randint
from math import exp,log
import time
from MNIST_HDR_GUI import input_data

data=pd.read_csv('train.csv')
testdata=pd.read_csv('test.csv')
x=data.drop('label',axis=1)
y=data['label']
ylin=y
z=np.zeros((28,28))
x=x.values
xtestf=testdata.values
m=40000
ylintest=np.array(ylin[m:int(1.5*m)])
ylintrain=np.array(y[0:m])


#visualising data 
def show(test,b):
    j=k=0
    for i in test[b]:
        z[j][k]=i
        k+=1
        if(k>27):
            k=0
            j=j+1
    img=Image.fromarray(z)
    img.show()
    
    
#converting linear output values to multiclass logical array
def lintoclass(y):
    out=np.zeros((len(y),10))
    for i in range(len(y)):
        ind=y[i]
        out[i][ind]=1
    y=out
    np.save('y.npy',y)

#loading already converted data
y=np.load('y.npy')
n=np.ones((m,1))


#splittong data
xtest=np.array(x[m:int(1.5*m)])
x=np.array(x[0:m])
ytest=np.array(y[m:int(1.5*m)])
nsize=len(xtest)
y=np.array(y[0:m])


#random initialisation
theta1=np.random.uniform(low=-0.013,high=0.013,size=(120,785))
theta2=np.random.uniform(low=-0.013,high=0.013,size=(10,121))
x=np.append(n,x,axis=1)


def loadweights():
    global theta1,theta2
    theta1=np.load('theta1.npy')
    theta2=np.load('theta2.npy')
#take 10 random values from test data and verify manually uncomment following lines to load already calculated theta weights
loadweights()

#argmax
def findmax(arr):
    maxi=0
    for i in range(len(arr)):
        if(maxi<arr[i]):
            maxi=arr[i]
            indi=i
    return(indi)


#forward propagation
def frwdprop():
    global a3
    z2=np.matmul(x,theta1.transpose())
    a2_ub=1/(1+np.exp(-z2))
    a2=np.append(n,a2_ub,axis=1)
    z3=np.matmul(a2,theta2.transpose())
    a3=1/(1+np.exp(-z3))
    return(a2_ub,a2)


#cost function and calculation of J(theta)
def cost(lam):
    Jtheta=(np.sum(np.matmul(-y,(np.log(a3)).transpose())-np.matmul((1-y),(np.log(1-a3)).transpose())))/len(x)+lam*(np.sum(theta1[:,1:]**2)+np.sum(theta2[:,1:]**2))/(2*len(x))
    return(Jtheta)


#Backpropagation
def backprop(lam):
    a2_ub,a2=frwdprop()
    d3=a3-y
    d2=(np.matmul(d3,theta2[:,1:])*((a2_ub)*(1-a2_ub)))
    Delta1=np.matmul(d2.transpose(),x)
    Delta2=np.matmul(d3.transpose(),a2)
    theta1_grad=Delta1/len(x)
    theta2_grad=Delta2/len(x)
    t1,t2=theta1,theta2
    t1[:,0]=t2[:,0]=0
    theta1_grad=theta1_grad+(lam/len(x))*t1
    theta2_grad=theta2_grad+(lam/len(x))*t2
    return(theta1_grad,theta2_grad)

#Gradient Descent
def gradientdescent(alpha):
    global theta1,theta2
    theta1_grad,theta2_grad=backprop(10)
    theta1=theta1-(alpha*theta1_grad)
    theta2=theta2-(alpha*theta2_grad)
    #Jtheta=cost(100)
iters=700

for i in range(iters):
    gradientdescent(0.1)
    


def saveweights():
    np.save('theta1.npy',theta1)
    np.save('theta2.npy',theta2)

    
#uncomment following lines to save your theta weights
#saveweights()


#actual prediction for manual testing
def predict(xam):
    xam=np.insert(xam,0,1)
    zp2=np.matmul(xam,theta1.transpose())
    ap2_ub=1/(1+np.exp(-zp2))
    ap2=np.insert(ap2_ub,0,1)
    zp3=np.matmul(ap2,theta2.transpose())
    ap3=1/(1+np.exp(-zp3))
    ans=findmax(ap3)
    #show(xtestf,0)
    print(ans)
    time.sleep(3)    

#train set vs test set accuracy
def trainvstest(xam):
    zp2=np.matmul(xam,theta1.transpose())
    ap2_ub=1/(1+np.exp(-zp2))
    ap2=np.insert(ap2_ub,0,1)
    zp3=np.matmul(ap2,theta2.transpose())
    ap3=1/(1+np.exp(-zp3))
    ans=findmax(ap3)
    return(ans)
counttrain=0
""""for i in range(m):
    ans=trainvstest(x[i])
    if(ans==ylintrain[i]):
        counttrain+=1
print("training accuracy =",(counttrain/m)*100)
counttest=0
xtest=np.append(n[:nsize],xtest,axis=1)
for i in range(nsize):
    ans=trainvstest(xtest[i])
    if(ans==ylintest[i]):
        counttest+=1
print("test accuracy =",(counttest/nsize)*100)
"""


"""for i in range(10):
    ind=randint(0,len(xtestf))
    predict(xtestf[ind])"""

#interactive testing using tkinter paint
image=input_data()
image=np.array(image)
image=image.flatten('C')
predict(image)